## Préparation du jeu de données PubChem - Distances absolues et relatives

Quels que soient les hyperparamètres des modèles que l'on entraîne (voir notebooks précédents), les réseaux apprennent très vite et sont bloqués dans leur apprentissage à environ 62% de la RMSE introduite par le bruit. Par exemple, lorsqu'on introduit un bruit de RMSE 170, le RMSE des prédictions des réseaux ne descend pas en dessous de 107, alors qu'on souhaiterait qu'il soit dans un ordre de grandeur situé entre 0 et 10.

La raison de ce manque de performances est peut-être le manque d'informations en entrée du réseau. Nous allons donc créér un jeu de données d'entrée contenant, en plus des distances aux points fixés du repère (les atomes fictifs), les distances inter-atomiques de la diagonale de largeur 4 de la matrice de distances, comme il était prévu dans la version initiale du réseau, avant que l'on ne réalise qu'il était impossible de reconstruire les positions des grosses molécules à partir de cette diagonale (voir notebook 2.0).

Ici, on va garder les distances aux points fixes du repère en sortie du réseau, on pourra donc reconstruire les positions des atomes des molécules si ce modèle s'avère efficace. Toutefois, nous allons en profiter pour créér un deuxième jeu d'étiquettes pour lequel le modèle devra prédire les distances inter-atomiques de la diagonale de la matrice de distances de largeur 4. Si le modèle prédisant les distances relatives s'avère plus efficace, alors il ne sera certes pas viable car on ne pourra pas reconstruire les molécules à partir de ses prédictions, mais cela indiquera que les mauvais résultats que nous avons jusque là sont dûs au format des données que nous attendons en sortie. 

Le bruit que nous allons introduire dans les jeux de données que nous allons préparer ici sera un bruit important (de RMSE 170), car il permettra d'évaluer facilement la qualité des modèles.


#### Chemin des fichiers

In [39]:
test_set_location = "../data/test_set_riken_v2.h5"
test_set_prepared_input_abs_rel_location = "../data/test_set_riken_v2_prepared_input_bruit+_abs_rel.h2"
test_set_labels_abs_location = "../data/test_set_riken_v2_labels_bruit+_abs.h2"
test_set_labels_rel_location = "../data/test_set_riken_v2_labels_bruit+_rel.h2"

train_set_location = "../data/train_set_riken_v2.h5"
train_set_prepared_input_abs_rel_location = "../data/train_set_riken_v2_prepared_input_bruit+_abs_rel.h5"
train_set_labels_abs_location = "../data/train_set_riken_v2_labels_bruit+_abs.h5"
train_set_labels_rel_location = "../data/train_set_riken_v2_labels_bruit+_rel.h5"

minimal_set_riken_location = "../data/minimal_set_riken_v2.h5"
minimal_set_prepared_input_abs_rel_location = "../data/minimal_set_riken_v2_prepared_input_bruit+_abs_rel.h5"
minimal_set_labels_abs_location = "../data/minimal_set_riken_v2_labels_bruit+_abs.h5"
minimal_set_labels_rel_location = "../data/minimal_set_riken_v2_labels_bruit+_rel.h5"

mini_set_riken_location = "../data/mini_set.h5"
mini_set_prepared_input_abs_rel_location = "../data/mini_set_prepared_input_bruit+_abs_rel.h5"
mini_set_labels_abs_location = "../data/mini_set_labels_bruit+_abs.h5"
mini_set_labels_rel_location = "../data/mini_set_labels_bruit+_rel.h5"


#### Définition de la fonction d'ajout de bruit

In [2]:
import numpy as np

def positions_bruitees(positions):    
    bruit = np.random.normal(loc=0.0, scale=0.1732, size=positions.shape)
    return ((positions + bruit), bruit)


#### Définition de la fonction de calcul de la matrice de distances aux points du repère fixes compressée à partir de la matrice des coordonnées des atomes

In [4]:
def matrice_distances_compr_abs(positions):
    """ Renvoie la matrice de distances compressée des positions des atomes passées en paramètres
    La matrice de distances compressée est définie de la façon suivante : pour chaque atome, on calcule
    la distance avec chaque point du repère. Une ligne i de la matrice (n,4) correspond aux distances
    de l'atome i avec chacun des quatre points du repère"""
    
    nb_at = len(positions)
    
    # On renvoie un tableau vide si la molécule est vide
    if nb_at == 0:
        return []
    
    repere = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])
    repere = np.vstack([repere]*nb_at)

    positions = np.tile(positions, 4).reshape(4*nb_at, 3)
    
    return np.sqrt(np.sum(np.power(positions-repere, 2), 1)).reshape(nb_at, 4)


#### Définition de la fonction de calcul de matrice de distances relatives (diagonale de largeur quatre)  partir des coordonnées des atomes

In [33]:
import math


def get_atome_coord(positions, index):
    """ Renvoie les coordonnées de l'atome ayant l'index donné """
    return (positions[index])
    
def calcul_distance(pt1, pt2):
    """ Renvoie la distance entre deux points représentés par leurs coordonnées (x, y, z) dans deux tableaux
    de forme (1, 3)"""
    return np.sqrt(np.sum(np.square(np.diff(np.array([pt1, pt2]), axis=0))))

def get_val_dist_matrice(i, j, positions):
    """ Renvoie la valeur de la matrice de distances aux coordonnées (i, j). La matrice complète n'est
    jamais calculée. """
    coord_i = get_atome_coord(positions, i)  # Coordonnées de l'atome i
    coord_j = get_atome_coord(positions, j)  # Coordonnées de l'atome j

    return calcul_distance(coord_i, coord_j)
    
def matrice_distances_compr_rel(positions):
    """ Renvoie la matrice de distances à partir des coordonnées des atomes et du nombre d'atomes dans
    la molécule. La matrice de distances complète n'est pas générée, seules les valeurs nécessaires
    sont calculées à la demande """
    
    nb_atomes = positions.shape[0]
    
    mat_distances = np.zeros(shape=(nb_atomes,4))
    
    for j in range(nb_atomes):
        k=0  # Permet d'accéder simplement à l'indice dans le tableau de sortie
        for i in range(j+1, j+5):
            if i < nb_atomes:
                mat_distances[j][k] = get_val_dist_matrice(i, j, positions)
            k += 1
            
    return mat_distances



#### Fonction de création des entrées et des labels des RN

In [126]:
import h5py
import time
import numpy as np

def creation_input_RN(set_location, input_rn_location_rel_abs, labels_location_abs,
                      labels_location_rel, epochs, nb_mol_mem):
    
    """ nb_mol_mem est le nombre maximal de molécules que l'on stocke simultanément en mémoire. Lorsque la 
    taille est atteinte, on écrit les entrées du RN et les cibles du RN sur le disque (taille max des
    batchs de traitement) 
    La nécessité d'utiliser des batchs de traitement en mémoire est due au fait que le traitement devient très
    long si on ne met pas de données en mémoire et qu'on écrit sur le disque à chaque itération. En utilisant
    les batchs, l'écriture sur le disque se fait en bloc.
    """
    
    start_time = time.time()    
    mol_vides = 0
    
    rmse_epochs_rel = []
    rmse_epochs_abs = []
    
    print("Creating input and label sets for "+set_location+" : ")

    # On charge le jeu de données original (en lecture seule)
    original_dataset_h5 = h5py.File(set_location, 'r')
    
    # On enregistre la taille du jeu de données
    taille = len(original_dataset_h5["anums"])
    
    # On créé les jeux de données d'entrée du RN et de labels
    input_rn_dataset_h5 = h5py.File(input_rn_location_rel_abs, 'w')
    labels_dataset_abs_h5 = h5py.File(labels_location_abs, 'w')
    labels_dataset_rel_h5 = h5py.File(labels_location_rel, 'w')

    
    # On créé les datasets inputs et targets 
    input_dataset = input_rn_dataset_h5.create_dataset("inputs", shape=(epochs*taille, 1800),
                                       dtype=np.float32, compression="gzip", 
                                       chunks=True)

    targets_dataset_abs = labels_dataset_abs_h5.create_dataset("targets", shape=(epochs*taille, 800),
                                       dtype=np.float32, compression="gzip", 
                                       chunks=True)
    
    targets_dataset_rel = labels_dataset_rel_h5.create_dataset("targets", shape=(epochs*taille, 800),
                                       dtype=np.float32, compression="gzip", 
                                       chunks=True)

    
    try:
        
        for k in range(epochs):
            
            print("Epoch "+str(k)+"...")

            # Contient les rmse de tous les bruits ajoutés à chaque molécule durant toute l'époque de génération
            # de l'entrée du RN. À la fin de l'époque, on en fait la moyenne et on l'ajoute à rmse_epochs
            bruits_epoch_rmse_abs = np.empty(shape=(taille,), dtype=np.float32)
            bruits_epoch_rmse_rel = np.empty(shape=(taille,), dtype=np.float32)
           
            input_coords = np.array(original_dataset_h5["riken_coords"])
            input_masses = np.array(original_dataset_h5["amasses"])
            
            # Indice de la première molécule traitée du batch courant
            indice_batch_courant = -nb_mol_mem
            
            # On parcourt toutes les molécules
            for i in range(taille):

                # On arrive au début d'un nouveau batch de traitement
                if i%nb_mol_mem == 0:
                    
                    # On écrit les données du batch précédent sur le disque s'il existe un batch précédent
                    if indice_batch_courant >= 0 :
                        print("Writing data to h5 for epoch "+str(k)+" and indexes from "+
                              str(indice_batch_courant)+" to "+str(indice_batch_courant+taille_batch_courant))
                        
                        input_dataset[k*taille+indice_batch_courant: 
                                      k*taille+indice_batch_courant+taille_batch_courant] = np_input_dataset
                        
                        targets_dataset_abs[k*taille+indice_batch_courant: 
                                            k*taille+indice_batch_courant+taille_batch_courant] = np_targets_dataset_abs
                        
                        targets_dataset_rel[k*taille+indice_batch_courant: 
                                            k*taille+indice_batch_courant+taille_batch_courant] = np_targets_dataset_rel
                        
                        
                    # On met à jour la première molécule du batch courant
                    indice_batch_courant += nb_mol_mem
                    
                    # On calcule la taille du nouveau batch
                    taille_batch_courant = min(nb_mol_mem, taille - i)
                    
                    print("Computing new batch...")
                        
                    # On créé les nouveaux tableaux en mémoire du batch courant
                    np_input_dataset = np.empty(shape=(taille_batch_courant, 1800))
                    np_targets_dataset_abs = np.empty(shape=(taille_batch_courant, 800))
                    np_targets_dataset_rel = np.empty(shape=(taille_batch_courant, 800))
                                   
                # On récupère les coordonnées de la molécule courante
                coords = np.array(input_coords[i]).reshape(-1,3)

                # On ajoute les positions des atomes fictifs aux coordonnées
                points_fictifs = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])
                coords = np.concatenate([points_fictifs, coords])
                
                # On calcule les matrices de distances aux points fixes et de distances relatives
                dist_init_abs = matrice_distances_compr_abs(coords)
                dist_init_rel = matrice_distances_compr_rel(coords)
                
                # On insère le bruit sur les coordonnées
                coords_bruit, bruit = positions_bruitees(coords)
                coords_bruit = coords_bruit.reshape(-1, 3)
                bruit = bruit.reshape(-1, 3)

                if len(coords) == 0:
                    mol_vides += 1
                else:
                    # On calcule les différence de distances cibles (en mÅ) et les distances bruitées (en Å),
                    # pour les distances relatives et absolues
                    dist_bruit_abs = matrice_distances_compr_abs(coords_bruit)
                    dist_bruit_rel = matrice_distances_compr_rel(coords_bruit)
                    delta_dist_targets_abs = (dist_init_abs - dist_bruit_abs)*1000
                    delta_dist_targets_rel = (dist_init_rel - dist_bruit_rel)*1000


                # On récupère les masses atomiques de la molécule courante
                masses = input_masses[i]

                # On initialise l'entrée du RN et le vecteur cible pour la molécule courante
                entree_courante = np.zeros(shape=(1800, 1))
                cible_courante_abs = np.zeros(shape=(200, 4))
                cible_courante_rel = np.zeros(shape=(200, 4))


                # On ajoute les coordonnées bruitées et les masses à l'entrée avec padding, et les coordonnées
                # cibles au dataset targets
                j=0
                for masse in masses:

                    # Ajout des distances absolues, des distances relatives et de la masse au vecteur entrée
                    index_input_courant = j*9
                    entree_courante[index_input_courant:index_input_courant+4] = dist_bruit_abs[j].reshape(4,1)
                    entree_courante[index_input_courant+4:index_input_courant+8] = dist_bruit_abs[j].reshape(4,1)
                    entree_courante[index_input_courant+8] = masse

                    # Ajout des données aux matrices cible
                    cible_courante_abs[j] = delta_dist_targets_abs[j]
                    cible_courante_rel[j] = delta_dist_targets_rel[j]

                    j+=1

                # On aplatit les matrices cible
                cible_courante_abs = cible_courante_abs.reshape(1, 800)
                cible_courante_rel = cible_courante_rel.reshape(1, 800)

                # On insère les données dans le tableau np en mémoire
                np_input_dataset[i%taille_batch_courant] = entree_courante.reshape(-1, 1800)
                np_targets_dataset_abs[i%taille_batch_courant] = cible_courante_abs
                np_targets_dataset_rel[i%taille_batch_courant] = cible_courante_rel

                # On ajoute le rmse ajouté à la molécule au tableau des rmse de l'époque
                np_delta_dist_targets_abs = np.array(delta_dist_targets_abs)
                np_delta_dist_targets_rel = np.array(delta_dist_targets_rel)
                bruits_epoch_rmse_abs[i] = np.sqrt(np.mean(np.square(np_delta_dist_targets_abs)))
                bruits_epoch_rmse_rel[i] = np.sqrt(np.mean(np.square(np_delta_dist_targets_rel)))
                
            
            
            print("end epoch")
            # On ajoute les données du dernier batch
            print("Writing data to h5 for epoch "+str(k)+" and indexes from "+
                              str(indice_batch_courant)+" to "+str(indice_batch_courant+taille_batch_courant))
                        
            input_dataset[k*taille+indice_batch_courant: 
                          k*taille+indice_batch_courant+taille_batch_courant] = np_input_dataset

            targets_dataset_abs[k*taille+indice_batch_courant: 
                                k*taille+indice_batch_courant+taille_batch_courant] = np_targets_dataset_abs

            targets_dataset_rel[k*taille+indice_batch_courant: 
                                k*taille+indice_batch_courant+taille_batch_courant] = np_targets_dataset_rel

                
            # On ajoute le rmse moyen de l'époque au tableau des rmse de toutes les époques
            rmse_epochs_abs.append(np.mean(bruits_epoch_rmse_abs))
            rmse_epochs_rel.append(np.mean(bruits_epoch_rmse_rel))
            
            
          

        print("Writing datasets to disk")
        input_rn_dataset_h5.flush()
        labels_dataset_abs_h5.flush()
        labels_dataset_rel_h5.flush()

        
        print("--- %s seconds ---" % (time.time() - start_time))
        
        # On calcule le rmse moyen de toutes les époques
        np_rmse_epochs_abs = np.array(rmse_epochs_abs)
        np_rmse_epochs_rel = np.array(rmse_epochs_rel)
        print("RMSE bruit moyen dist absolues : "+str(np.mean(np_rmse_epochs_abs)))
        print("RMSE bruit moyen dist relatives : "+str(np.mean(np_rmse_epochs_rel)))

        
    finally:
        original_dataset_h5.close()
        input_rn_dataset_h5.close()
        labels_dataset_abs_h5.close()
        labels_dataset_rel_h5.close()

    



#### Préparation des données d'entrée du RN et des labels pour le jeu minimal

In [ ]:
creation_input_RN(minimal_set_riken_location, minimal_set_prepared_input_abs_rel_location,
                  minimal_set_labels_abs_location, minimal_set_labels_rel_location, 3, 500000)

##### Sortie

```
Creating input and label sets for ../data/minimal_set_riken_v2.h5 :
Epoch 0...
Computing new batch...
end epoch
Writing data to h5 for epoch 0 and indexes from 0 to 100000
Epoch 1...
Computing new batch...

end epoch
Writing data to h5 for epoch 1 and indexes from 0 to 100000
Epoch 2...
Computing new batch...
end epoch
Writing data to h5 for epoch 2 and indexes from 0 to 100000
Writing datasets to disk
--- 2636.2060313224792 seconds ---
RMSE bruit moyen dist absolues : 177.54662
RMSE bruit moyen dist relatives : 233.56984

```

#### Préparation des données d'entrée du RN et des labels pour le jeu d'entraînement


In [134]:
creation_input_RN(train_set_location, train_set_prepared_input_abs_rel_location,
                  train_set_labels_abs_location, train_set_labels_rel_location, 3, 500000)

Creating input and label sets for ../data/train_set_riken_v2.h5 : 


OSError: Unable to open file (unable to open file: name = '../data/train_set_riken_v2.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

##### Sortie

```
Creating input and label sets for ../data/train_set_riken_v2.h5 :
Epoch 0...
Computing new batch...
Writing data to h5 for epoch 0 and indexes from 0 to 500000
Computing new batch...
Writing data to h5 for epoch 0 and indexes from 500000 to 1000000
Computing new batch...
Writing data to h5 for epoch 0 and indexes from 1000000 to 1500000
Computing new batch...
Writing data to h5 for epoch 0 and indexes from 1500000 to 2000000
Computing new batch...
Writing data to h5 for epoch 0 and indexes from 2000000 to 2500000
Computing new batch...
Writing data to h5 for epoch 0 and indexes from 2500000 to 3000000
Computing new batch...
end epoch
Writing data to h5 for epoch 0 and indexes from 3000000 to 3309620
Epoch 1...
Computing new batch...
Writing data to h5 for epoch 1 and indexes from 0 to 500000
Computing new batch...
Writing data to h5 for epoch 1 and indexes from 500000 to 1000000
Computing new batch...
Writing data to h5 for epoch 1 and indexes from 1000000 to 1500000
Computing new batch...
Writing data to h5 for epoch 1 and indexes from 1500000 to 2000000
Computing new batch...
Writing data to h5 for epoch 1 and indexes from 2000000 to 2500000
Computing new batch...
Writing data to h5 for epoch 1 and indexes from 2500000 to 3000000
Computing new batch...
end epoch
Writing data to h5 for epoch 1 and indexes from 3000000 to 3309620
Epoch 2...
Computing new batch...
Writing data to h5 for epoch 2 and indexes from 0 to 500000
Computing new batch...
Writing data to h5 for epoch 2 and indexes from 500000 to 1000000
Computing new batch...
Writing data to h5 for epoch 2 and indexes from 1000000 to 1500000
Computing new batch...
Writing data to h5 for epoch 2 and indexes from 1500000 to 2000000
Computing new batch...
Writing data to h5 for epoch 2 and indexes from 2000000 to 2500000
Computing new batch...
Writing data to h5 for epoch 2 and indexes from 2500000 to 3000000
Computing new batch...
end epoch
Writing data to h5 for epoch 2 and indexes from 3000000 to 3309620
Writing datasets to disk
--- 71815.83514904976 seconds ---
RMSE bruit moyen dist absolues : 177.56111
RMSE bruit moyen dist relatives : 233.6153
```


#### Préparation des données d'entrée du RN et des labels pour le jeu de test


In [ ]:
creation_input_RN(test_set_location, test_set_prepared_input_abs_rel_location,
                  test_set_labels_abs_location, test_set_labels_rel_location, 3, 500000)

##### Sortie
```
Creating input and label sets for ../data/test_set_riken_v2.h5 :
Epoch 0...
Computing new batch...
end epoch
Writing data to h5 for epoch 0 and indexes from 0 to 367736
Epoch 1...
Computing new batch...
end epoch
Writing data to h5 for epoch 1 and indexes from 0 to 367736
Epoch 2...
Computing new batch...
end epoch
Writing data to h5 for epoch 2 and indexes from 0 to 367736
Writing datasets to disk
--- 7922.944814920425 seconds ---
RMSE bruit moyen dist absolues : 177.58287
RMSE bruit moyen dist relatives : 233.66093

```